### Reading and Processing call data

In [2]:
import pandas as pd
from pathlib import Path
import os
os.chdir('/Users/deepakm/mywork/data-prod')
# os.getcwd()

In [3]:
datapath1 = Path('./data/json150')
datapath2 = Path('./data/JSON 115 files - Apr 1')

In [4]:
print(datapath2)
file_list1= list(datapath2.glob('*.json'))
file_list2= list(datapath2.glob('*.json'))
file_list = file_list1 + file_list2
len(file_list),file_list[0]

data/JSON 115 files - Apr 1


(188, PosixPath('data/JSON 115 files - Apr 1/crhg_2023_Dec_04_14_46_53.json'))

In [5]:
import json
json_data=[]
good_file=[]
bad_file=[]
for file in file_list:
    try:
        # print(file)
        f = open(file)
        data = json.load(f)
        json_data.append(data)
        good_file.append(file)
    except:
        bad_file.append(file)
    f.close()

print(len(json_data))
print(f"Good files: {len(good_file)}, Bad files: {len(bad_file)}")
# json_data

102
Good files: 102, Bad files: 86


### Combine all data to dataframe

In [33]:
df=pd.json_normalize(json_data)
df['Timestamp']=df['Timestamp'].str.replace("_","-",2)
df['Timestamp']=df['Timestamp'].str.replace("_"," ",1)
df['Timestamp']=df['Timestamp'].str.replace("_",":",2)
df['Timestamp']=pd.to_datetime(df['Timestamp'],format='mixed')
df['Timestamp']=pd.to_datetime(df['Timestamp']).dt.normalize()
df['call_weekday'] = df['Timestamp'].dt.day_name()
df['call_month'] = df['Timestamp'].dt.month_name()
df['CallerContent']=df.loc[:,(df.columns.str.endswith('CallerContent'))].apply(lambda x: ' '.join(x.dropna().astype(str).values),axis=1)
df['ReceiverContent']=df.loc[:,(df.columns.str.endswith('ReceiverContent'))].apply(lambda x: ' '.join(x.dropna().astype(str).values),axis=1)

print(df.isna().sum())
print(df[['CallerContent','ReceiverContent']])
df.sample(10)

Metadata                                  2
Caller                                   40
Receiver                                  0
Timestamp                                 0
CallContent.Seq1.CallerContent           70
                                       ... 
CallContent.Seq3.OtherSpeakerContent    100
call_weekday                              0
call_month                                0
CallerContent                             0
ReceiverContent                           0
Length: 182, dtype: int64
                                         CallerContent  \
0    last known thanks for calling corruption from ...   
1    Hi, I'm calling to talk to Brandy. This is Car...   
2    Hi, Dominique. This is Sandra Arrington. I jus...   
3    Hi, Dominique. This is Judith again. Can you t...   
4    Hi, Dominic. I'm trying to reach the office an...   
..                                                 ...   
97   Hi, I need to speak with someone regarding a r...   
98   Yeah, this is Alan Sm

,Metadata,Caller,Receiver,Timestamp,CallContent.Seq1.CallerContent,CallContent.Seq1.ReceiverContent,CallContent.Seq1.Timestamp,CallContent.Seq2.CallerContent,CallContent.Seq2.ReceiverContent,CallContent.Seq2.Timestamp,...,CallContent.Seq34.CallerContent,CallContent.Seq35.CallerContent,CallContent.Seq36.CallerContent,OtherSpeaker.Content,OtherSpeaker.Timestamp,CallContent.Seq3.OtherSpeakerContent,call_weekday,call_month,CallerContent,ReceiverContent
19,value,Allison,Kumar,2023-10-26,"Hi, yes. I'd like to make an appointment with ...",Good morning. Thanks for calling Carlson Famil...,2023-10-26 10:04:16,A televisit would be fine.,Would you like to meet the doctor in person or...,2023-10-26 10:04:25,...,NaN,NaN,NaN,NaN,NaN,NaN,Thursday,October,"Hi, yes. I'd like to make an appointment with ...",Good morning. Thanks for calling Carlson Famil...
81,value,Elizabeth,Kumaran (Crofton Family Practice),2023-12-04,Hello?,"Good afternoon, this is Kumaran from Family Pr...",2023_12_04_14:51:12,"Yes, I'd like to make an appointment for follo...",Sure. Would you like to meet the doctor in per...,2023_12_04_14:51:12,...,NaN,NaN,NaN,NaN,NaN,NaN,Monday,December,"Hello? Yes, I'd like to make an appointment fo...","Good afternoon, this is Kumaran from Family Pr..."
69,value,NaN,Dominic from Grobson Family Practice,2023-09-21,NaN,Good afternoon. Thank you for calling Grobson ...,2023-09-21 00:00:00,"Hi, this is Karen Sutton. I have an appointmen...",NaN,2023-09-21 00:06:00,...,NaN,NaN,NaN,NaN,NaN,NaN,Thursday,September,"Hi, this is Karen Sutton. I have an appointmen...",Good afternoon. Thank you for calling Grobson ...
96,value,Ed Jones,Dominic (Crofton Family Practice),2023-12-04,"Hi, this is Ed Jones. I'm a patient at Dr. Wil...",Good afternoon. Thanks for calling Crofton Fam...,2023_12_04_14:59:14,Okay. Thank you.,"Sure. Regarding that, I need to transfer you t...",2023_12_04_14:59:14,...,NaN,NaN,NaN,NaN,NaN,NaN,Monday,December,"Hi, this is Ed Jones. I'm a patient at Dr. Wil...",Good afternoon. Thanks for calling Crofton Fam...
39,value,NaN,Kathy from Crofton Family Practice,2023-09-21,NaN,Thank you for calling Crofton Family Practice....,2023-09-21 11:01:33,"Hi, this is Amanda from Luminous Health Imagin...",Would you like to speak to the nurse?,2023-09-21 11:01:34,...,NaN,NaN,NaN,Good morning.,2023-09-21 11:01:33,Sure.,Thursday,September,"Hi, this is Amanda from Luminous Health Imagin...",Thank you for calling Crofton Family Practice....
41,value,Anonymous,Kathy,2023-09-20,NaN,Good afternoon. Thank you for calling Crosstow...,2023-09-20 16:24:47,Kathy? Yes. Is Brandy there? Would you like to...,Yeah. Kindly stay online. Let me transfer the ...,2023-09-20 16:24:52,...,NaN,NaN,NaN,NaN,NaN,NaN,Wednesday,September,Kathy? Yes. Is Brandy there? Would you like to...,Good afternoon. Thank you for calling Crosstow...
59,value,NaN,Kathy from Proud and Family Practice,2023-09-21,NaN,Good morning. Thank you for calling Proud and ...,2023-09-21 00:01:00,"Kathy, hi. I emailed my pre-op paperwork over ...","Yeah, kindly give me a minute.",2023-09-21 00:06:00,...,NaN,NaN,NaN,NaN,NaN,NaN,Thursday,September,"Kathy, hi. I emailed my pre-op paperwork over ...",Good morning. Thank you for calling Proud and ...
71,value,Anonymous,Dominic,2023-09-21,NaN,Good morning. Thank you for calling Crofton Fa...,2023-09-21 09:18:00,"Yeah, hi. This is Jim Neiman. How are you today?",So thank you for asking what you're doing toda...,2023-09-21 09:18:04,...,NaN,NaN,NaN,NaN,NaN,NaN,Thursday,September,"Yeah, hi. This is Jim Neiman. How are you toda...",Good morning. Thank you for calling Crofton Fa...
54,value,Judith,Dominique,2023-11-01,"Hi, Dominique. This is Judith again. Can you t...","Okay. Kindly give me a minute, ma'am. Let me t...",2023-11-01 14:29:13,You too.,Hello. You have reached the nurse line. If thi...,2023-11-01 14:29:17,...,NaN,NaN,NaN,NaN,NaN,NaN,Wednesday,November,"Hi, Dominique. This is Judith again. Can you t...","Okay. Kindly give me a minute, ma'am. Let me 

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102 entries, 0 to 101
Columns: 182 entries, Metadata to ReceiverContent
dtypes: datetime64[ns](1), object(181)
memory usage: 145.2+ KB


In [ ]:
print(len (df.columns),df.columns)

181 Index(['Metadata', 'Caller', 'Receiver', 'Timestamp',
       'CallContent.Seq1.CallerContent', 'CallContent.Seq1.ReceiverContent',
       'CallContent.Seq1.Timestamp', 'CallContent.Seq2.CallerContent',
       'CallContent.Seq2.ReceiverContent', 'CallContent.Seq2.Timestamp',
       ...
       'CallContent.Seq32.CallerContent', 'CallContent.Seq34.CallerContent',
       'CallContent.Seq35.CallerContent', 'CallContent.Seq36.CallerContent',
       'OtherSpeaker.Content', 'OtherSpeaker.Timestamp',
       'CallContent.Seq3.OtherSpeakerContent', 'Time_Start', 'CallerContent',
       'ReceiverContent'],
      dtype='object', length=181)


### Number of Calls by Caller and Receiver Name

In [34]:
print(df.Caller.value_counts())
print(df.Receiver.value_counts())


Caller
Anonymous                   8
Judith                      4
no_name                     2
Elizabeth                   2
Alan                        2
Aquanetta Delaney           2
Ed Jones                    2
Stephen Chapman             2
Ellen                       2
Patrick Ferguson            2
No Name (Beverly Cantor)    2
Mr. Leatherman              2
Dr. Williams                2
Roberta Nelson              2
Robert                      2
Unnamed                     2
Sandra Arrington            2
Ms. Watson                  2
Emily                       2
Chris                       2
Allison                     2
No Name                     2
Madonna Simon               2
Lisa Cunningham             2
Alan Smith                  2
Camelia Watson              2
Unknown                     2
Name: count, dtype: int64
Receiver
Dominic                                  32
Dominic from Crofton Family Practice      8
Kumar                                     6
Dominique       

### Sentiment Analysis on Caller Content

In [36]:
# pip install transformers
from transformers import pipeline
sentiment_model = pipeline("sentiment-analysis", truncation=True, max_length=512, model='distilbert/distilbert-base-uncased-finetuned-sst-2-english')
data = ["I love you", "I hate you"]
sentiment_model(data)

[{'label': 'POSITIVE', 'score': 0.9998656511306763},
 {'label': 'NEGATIVE', 'score': 0.9991129040718079}]

In [37]:
# Sentiment Analysis of Caller Data
df['sent_model']= df['CallerContent'].apply(sentiment_model) 
df['Caller_Sentiment']= df['sent_model'].apply(lambda s: s[0]['label'])
df['Caller_Sentiment_Score']= df['sent_model'].apply(lambda s: s[0]['score'])
print(df[['CallerContent','Caller_Sentiment','Caller_Sentiment_Score']])

                                         CallerContent Caller_Sentiment  \
0    last known thanks for calling corruption from ...         NEGATIVE   
1    Hi, I'm calling to talk to Brandy. This is Car...         NEGATIVE   
2    Hi, Dominique. This is Sandra Arrington. I jus...         POSITIVE   
3    Hi, Dominique. This is Judith again. Can you t...         NEGATIVE   
4    Hi, Dominic. I'm trying to reach the office an...         NEGATIVE   
..                                                 ...              ...   
97   Hi, I need to speak with someone regarding a r...         POSITIVE   
98   Yeah, this is Alan Smith again. Listen, you we...         NEGATIVE   
99   Hi, this is Barbara Wondell. I'm at Urgent Car...         POSITIVE   
100  Hi, yes. Good morning. I received a bill in th...         NEGATIVE   
101  Hi, Brandi. This is Judith. I'm calling from D...         POSITIVE   

     Caller_Sentiment_Score  
0                  0.991963  
1                  0.979249  
2        

### Emotion Analysis on Caller Content

In [38]:
# model= "pysentimiento/robertuito-sentiment-analysis"
model="bhadresh-savani/distilbert-base-uncased-emotion"
emotion_model = pipeline(model=model, truncation=True, max_length=512)
emotion_model(data)


[{'label': 'love', 'score': 0.9608992338180542},
 {'label': 'anger', 'score': 0.826848566532135}]

In [39]:
# Emotional Analysis of Caller Data
df['emot_model']= df['CallerContent'].apply(emotion_model) 
df['Caller_Emotion']= df['emot_model'].apply(lambda s: s[0]['label'])
df['Caller_Emotion_Score']= df['emot_model'].apply(lambda s: s[0]['score'])
print(df[['CallerContent','Caller_Emotion','Caller_Emotion_Score']])

                                         CallerContent Caller_Emotion  \
0    last known thanks for calling corruption from ...            joy   
1    Hi, I'm calling to talk to Brandy. This is Car...            joy   
2    Hi, Dominique. This is Sandra Arrington. I jus...            joy   
3    Hi, Dominique. This is Judith again. Can you t...          anger   
4    Hi, Dominic. I'm trying to reach the office an...            joy   
..                                                 ...            ...   
97   Hi, I need to speak with someone regarding a r...            joy   
98   Yeah, this is Alan Smith again. Listen, you we...            joy   
99   Hi, this is Barbara Wondell. I'm at Urgent Car...            joy   
100  Hi, yes. Good morning. I received a bill in th...        sadness   
101  Hi, Brandi. This is Judith. I'm calling from D...            joy   

     Caller_Emotion_Score  
0                0.985529  
1                0.953509  
2                0.997964  
3          

In [14]:
df.head()

,Metadata,Caller,Receiver,Timestamp,CallContent.Seq1.CallerContent,CallContent.Seq1.ReceiverContent,CallContent.Seq1.Timestamp,CallContent.Seq2.CallerContent,CallContent.Seq2.ReceiverContent,CallContent.Seq2.Timestamp,...,CallContent.Seq3.OtherSpeakerContent,Time_Start,CallerContent,ReceiverContent,sent_model,Caller_Sentiment,Caller_Sentiment_Score,emot_model,Caller_Emotion,Caller_Emotion_Score
0,value,no_name,Crafton Family Practice (dominic),2023-Dec-04 14:46:53,last known thanks for calling corruption from ...,is this you for the day hi I was wondering if ...,2023_Dec_04_14:46:53,pardon me yeah can you give me a minute let me...,sure I'm sorry yeah I'm really sorry mom all t...,2023_Dec_04_14:46:53,...,NaN,2023-12-04 14:46:53,last known thanks for calling corruption from ...,is this you for the day hi I was wondering if ...,"[{'label': 'NEGATIVE', 'score': 0.991962909698...",NEGATIVE,0.991963,"[{'label': 'joy', 'score': 0.9855291247367859}]",joy,0.985529
1,value,NaN,Kumar from Crofton Family Practice,2023-09-21 13:50:56,NaN,Good afternoon. Thank you for calling Crofton ...,2023-09-21 00:00:00,"Hi, I'm calling to talk to Brandy. This is Car...","Okay, let me transfer you to Brandy, and she c...",2023-09-21 00:05:00,...,NaN,2023-09-21 13:50:56,"Hi, I'm calling to talk to Brandy. This is Car...",Good afternoon. Thank you for calling Crofton ...,"[{'label': 'NEGATIVE', 'score': 0.979249179363...",NEGATIVE,0.979249,"[{'label': 'joy', 'score': 0.9535087943077087}]",joy,0.953509
2,value,Sandra Arrington,Dominique,2023-09-20 16:08:10,NaN,Good afternoon. Thank you for calling Crofton ...,2023-09-20 16:08:10,"Hi, Dominique. This is Sandra Arrington. I jus...","Okay. Regarding that, let me transfer the call...",2023-09-20 16:08:14,...,NaN,2023-09-20 16:08:10,"Hi, Dominique. This is Sandra Arrington. I jus...",Good afternoon. Thank you for calling Crofton ...,"[{'label': 'POSITIVE', 'score': 0.999589860439...",POSITIVE,0.999590,"[{'label': 'joy', 'score': 0.997963547706604}]",joy,0.997964
3,value,Judith,Dominique,2023-11-01 14:29:13,"Hi, Dominique. This is Judith again. Can you t...","Okay. Kindly give me a minute, ma'am. Let me t...",2023-11-01 14:29:13,You too.,Hello. You have reached the nurse line. If thi...,2023-11-01 14:29:17,...,NaN,2023-11-01 14:29:13,"Hi, Dominique. This is Judith again. Can you t...","Okay. Kindly give me a minute, ma'am. Let me t...","[{'label': 'NEGATIVE', 'score': 0.986896097660...",NEGATIVE,0.986896,"[{'label': 'anger', 'score': 0.650580644607544}]",anger,0.650581
4,value,Camelia Watson,Dominic,2023-09-20 15:21:05,"Hi, Dominic. I'm trying to reach the office an...","Okay. Kindly give me a minute, ma'am. Let me t...",2023-09-20 15:21:05,NaN,NaN,2023-09-20 15:21:18,...,NaN,2023-09-20 15:21:05,"Hi, Dominic. I'm trying to reach the office an...","Okay. Kindly give me a minute, ma'am. Let me t...","[{'label': 'NEGATIVE', 'score': 0.551674485206...",NEGATIVE,0.551674,"[{'label': 'joy', 'score': 0.9758375287055969}]",joy,0.975838


In [15]:
df.shape, df.columns


((102, 187),
 Index(['Metadata', 'Caller', 'Receiver', 'Timestamp',
        'CallContent.Seq1.CallerContent', 'CallContent.Seq1.ReceiverContent',
        'CallContent.Seq1.Timestamp', 'CallContent.Seq2.CallerContent',
        'CallContent.Seq2.ReceiverContent', 'CallContent.Seq2.Timestamp',
        ...
        'CallContent.Seq3.OtherSpeakerContent', 'Time_Start', 'CallerContent',
        'ReceiverContent', 'sent_model', 'Caller_Sentiment',
        'Caller_Sentiment_Score', 'emot_model', 'Caller_Emotion',
        'Caller_Emotion_Score'],
       dtype='object', length=187))

In [28]:
# df['Timestamp']=pd.to_datetime(df['Timestamp'],format='mixed')
# df['Timestamp2']=pd.to_datetime(df['Timestamp2']).dt.normalize()
# df['call_weekday'] = df['Timestamp'].dt.day_name()
df.groupby(df.call_weekday).count()

,Metadata,Caller,Receiver,Timestamp,CallContent.Seq1.CallerContent,CallContent.Seq1.ReceiverContent,CallContent.Seq1.Timestamp,CallContent.Seq2.CallerContent,CallContent.Seq2.ReceiverContent,CallContent.Seq2.Timestamp,...,Time_Start,CallerContent,ReceiverContent,sent_model,Caller_Sentiment,Caller_Sentiment_Score,emot_model,Caller_Emotion,Caller_Emotion_Score,Timestamp2
call_weekday,,,,,,,,,,,,,,,,,,,,,
Monday,16,16,16,16,16,16,16,16,16,16,...,16,16,16,16,16,16,16,16,16,16
Thursday,56,18,58,58,2,56,58,52,52,58,...,58,58,58,58,58,58,58,58,58,58
Wednesday,28,28,28,28,14,28,28,24,26,28,...,28,28,28,28,28,28,28,28,28,28


In [30]:
df['call_weekday'].unique()

array(['Monday', 'Thursday', 'Wednesday'], dtype=object)

In [32]:
df.describe()

,Timestamp,Time_Start,Caller_Sentiment_Score,Caller_Emotion_Score,Timestamp2
count,102,102,102.000000,102.000000,102
mean,2023-10-05 05:16:26.294117376,2023-10-05 05:16:26.294117376,0.909860,0.897566,2023-10-04 16:00:00
min,2023-09-20 15:10:46,2023-09-20 15:10:46,0.530403,0.387061,2023-09-20 00:00:00
25%,2023-09-21 08:59:28.249999872,2023-09-21 08:59:28.249999872,0.863827,0.908931,2023-09-21 00:00:00
50%,2023-09-21 11:28:49,2023-09-21 11:28:49,0.988718,0.985529,2023-09-21 00:00:00
75%,2023-09-21 14:36:45.500000,2023-09-21 14:36:45.500000,0.997472,0.996172,2023-09-21 00:00:00
max,2023-12-04 16:07:23,2023-12-04 16:07:23,0.999823,0.998739,2023-12-04 00:00:00
std,NaN,NaN,0.143468,0.167785,NaN


### Reading data records as rows for each sequence of call

In [ ]:
def files_df(file_list):
    dfl = []
    for file in file_list:
        df=pd.read_json(file).reset_index()
        df['data_file']=file.stem
        df['Timestamp']=df['Timestamp'].astype(str)
        df['Timestamp']=df['Timestamp'].str.replace("_","-",2)
        df['Timestamp']=df['Timestamp'].str.replace("_"," ",1)
        df['Timestamp']=df['Timestamp'].str.replace("_",":",2)
        df['Timestamp2']=pd.to_datetime(df['Timestamp'],format='mixed')
        df['Timestamp']=pd.to_datetime(df['Timestamp']).dt.normalize()
        # df.rename(columns={'Timestamp':'Time_Start'},inplace=True)
        # df['CallerContent'] = df.CallContent['CallerContent']
        dfl.append(df)
    df= pd.concat(dfl)
    return df

df = files_df(good_file)
df.head()

In [ ]:
print(df['CallContent'].apply(pd.Series).columns)
# df['CallContent'].apply(lambda x: pd.Series(x).get('CallerContent'))
# df['CallContent'].apply(lambda x: pd.Series(x).get('ReceiverContent'))
# df['CallContent'].apply(lambda x: pd.Series(x).get('OtherSpeakerContent'))
# df['CallContent'].apply(lambda x: pd.Series(x).get('SpeakerContent'))
# df['CallContent'].apply(lambda x: pd.Series(x).get('BrandiContent'))
# df['CallContent'].apply(lambda x: pd.Series(x).get(0))

# df[(df['CallContent'].apply(lambda x: pd.Series(x).get('BrandiContent')).notna())]
df['CallContent'].apply(pd.Series)
# df['CallContent'].apply(pd.Series).isna().sum() #['0']
# to_csv('call_content.csv', index=False)
# df.head()

In [ ]:
df.info()

In [ ]:
# url='https://drive.google.com/drive/u/0/folders/1Mtkwn7pwB54C3DN14Gzr8Yozkjl0rVKP'
# pd.read_json('https://drive.usercontent.google.com/download?id={}&export=download&authuser=0&confirm=t'.format(url.split('/')[-2]))
# url.split('/')[-1]

In [ ]:
# url='https://drive.google.com/drive/u/0/folders/1Mtkwn7pwB54C3DN14Gzr8Yozkjl0rVKP'
# file_id=url.split('/')[-1]
# dwn_url='https://drive.google.com/uc?id=' + file_id
# df = pd.read_json(dwn_url)
# print(df.head())